# cadCAD Experiment
Over 500 timesteps, money comes in to the system each timestep and is distributed to delegators according to the proportion of shares they have bought.  The share price and number are tied to a bonding curve.  

They purchase shares in the system according to their belief of the future revenue streams.

In [ ]:
# if this crashes, run this:
# pip install ipython-autotime
%load_ext autotime
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

In [ ]:
import matplotlib.pyplot as plt
from model.run import run

# df = run()


In [ ]:
from cadCAD_tools import profile_run

from model.psub import psubs
from model.state import genesis_state
from model.config import simulation_config
from model.config import params

TIMESTEPS = len(simulation_config[0]['T'])
SAMPLES = simulation_config[0]['N'] 
# df = run()
df = profile_run(genesis_state,
                 params,
                 psubs,
                 TIMESTEPS,
                 SAMPLES)
from cadCAD_tools.profiling.visualizations import visualize_substep_impact

visualize_substep_impact(df, relative=True)


In [ ]:
max_substep = max(df.substep)
print(max_substep)
df = df[df.substep==max_substep].reset_index()

In [ ]:
df

# Revenue coming in each period.  
### Mean is an exponentially distributed parameter.  

## Shock to the system @ timestep 250.
### Timsteps 1-250: mean=7
### Timesteps 251-500: mean=70



In [ ]:
df.period_revenue.plot()

In [ ]:
import pandas as pd
df1 = df.reset_index().delegators

delegator_revenue_token_holdings = {}
delegator_shares = {}
timestep = 0
for delegator_dict in df1:
    for delegator_id, delegator in delegator_dict.items():
        if delegator_id not in delegator_revenue_token_holdings:
            delegator_revenue_token_holdings[delegator_id] = {}
            delegator_shares[delegator_id] = {}
        delegator_revenue_token_holdings[delegator_id][timestep] = delegator.revenue_token_holdings
        delegator_shares[delegator_id][timestep] = delegator.shares
        
    timestep += 1

# Shares broken down by Delegator
This shows a stacked plot with the sum of shares owned by the individual delegators. There is a jump in shares at the shock point at timestep = 250 where the dividend value from owning the shares increases 10x.

<table class="tg">
<thead>
  <tr>
    <th>Delegator ID</th>
    <th>Delegator Type</th>
    <th>Delegator Pricing Model</th>
    <th>Delegator Description</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>0</td>
    <td>Owner</td>
    <td>Value</td>
    <td>spot_price will converge to the value of the revenue stream.</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Non-Owner</td>
    <td>Regression to Mean</td>
    <td>spot_price will return to what it used to be.</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Non-Owner</td>
    <td>Value</td>
    <td>spot_price will converge to the value of the revenue stream.</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Non-Owner</td>
    <td>Trendline</td>
    <td>spot_price will continue moving in the same direction as it has in the past.</td>
  </tr>
</tbody>
</table>


#### NOTE: The delegators act in randomized order, so 0 and 2 will not have the same result.

In [ ]:


df_plot = pd.DataFrame(delegator_shares)
df_plot.plot(kind='area')
plt.gca().legend(loc=9, bbox_to_anchor=(2.5, 1), ncol=12)
plt.title('Shares Owned By Delegators')
plt.xlabel('Epoch Number')
plt.ylabel('Delegator Shares')

# Cumulative Revenue Token Holdings



In [ ]:
df_plot = pd.DataFrame(delegator_revenue_token_holdings)
df_plot.plot(kind='area')
plt.gca().legend(loc=9, bbox_to_anchor=(2.5, 1), ncol=12)
plt.title('Cumulative Revenue Token Holdings')
plt.xlabel('Epoch Number')
plt.ylabel('Revenue Token Holdings')

Exploratory data analysis

In [ ]:
df['shareholders'] = df.delegators.apply(lambda x: [k for k in x.keys() if x[k].shares>0] )

In [ ]:
df['shares'] = df.delegators.apply(lambda x: [x[k].shares for k in x.keys() if x[k].shares>0] )

In [ ]:
df_list = []
for t in range(len(df)):
    temp_df = pd.DataFrame({'shareholder':df.shareholders.iloc[t] , 'shares':df.shares.iloc[t]})
    temp_df['Epoch Number'] = t

    df_list.append(temp_df)

shareholder_df = pd.concat(df_list)

In [ ]:
shareholder_df

# Price per share

In [ ]:
df.spot_price.plot()

In [ ]:
df['period_revenue_per_share'] = df.period_revenue/df.supply

# Revenue per share 

In [ ]:
df.period_revenue_per_share.plot()

# Estimated Share Valuation

In [ ]:
#estimated share valuation
risk_adj = .7 #if agents get their own param then there exists an estimated value of this system level param
discount_factor = .95 #if agents get their own param there exist an estimated value of this systel level param

df['share_valuation'] = df.spot_price * risk_adj + df.period_revenue_per_share / (1 - discount_factor)
df['ewm_share_valuation'] = df.share_valuation.ewm(halflife = 10).mean()

In [ ]:
df[['share_valuation', 'spot_price', 'ewm_share_valuation', 'timestep']].plot(x='timestep', color='ybg')

In [ ]:
df.columns

In [ ]:
# each timestep has a dict with {delegatorId: private_price} 
df['private_prices'] = df.delegators.apply(lambda x: {k: x[k].private_price for k in x.keys()} )
df['regression_to_mean_private_prices'] = df.delegators.apply(lambda x: {k: x[k].regression_to_mean_private_price for k in x.keys()} )
df['value_private_prices'] = df.delegators.apply(lambda x: {k: x[k].value_private_price for k in x.keys()} )
df['trendline_private_prices'] = df.delegators.apply(lambda x: {k: x[k].trendline_private_price for k in x.keys()} )


In [ ]:
import matplotlib.pylab as plt

# d = df['regression_to_mean_private_prices']
# d = df['value_private_prices']
d = df['private_prices']

timestep = 0
delegateId = 0

# initialize a list of 4 lists
x = [[] for _ in range(4)]
y = [[] for _ in range(4)]

fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2,2)
axs = [ax0, ax1, ax2, ax3]

for timestep in range(500):
    # print(f'{timestep=}')
    for delegateId, price in d[timestep].items():    
        # print(f'{delegateId=}, {price=}')
        # The data has to be in the form x = [timesteps], y = [values]
        # the data is in the form y = dict({key=delegator, value=private_price})        
        x[delegateId].append(timestep)
        y[delegateId].append(price)
        
colors = 'rgby'
for delegateId in range(4):    
    
    axs[delegateId].plot(x[delegateId], y[delegateId], colors[delegateId])
    if delegateId in (0,2):
        axs[delegateId].set_title(f'{delegateId=}, Value')
    elif delegateId == 1:
        axs[delegateId].set_title(f'{delegateId=}, Regression to Mean')
    elif delegateId == 3:
        axs[delegateId].set_title(f'{delegateId=}, Trendline')


plt.setp(ax0.get_xticklabels(), visible=False)
plt.setp(ax1.get_xticklabels(), visible=False)
plt.show()


# Questions?
* What other parameters can we add to the model?  
* What other visualizations would we like to see?

In [ ]:
# df['regression_to_mean_private_prices']

# for i in range(500):
#     # print(df['regression_to_mean_private_prices'][i])
#     # print(df['value_private_prices'][i])
#     # print(df['trendline_private_prices'][i])
#     print(i, df['private_prices'][i])

In [ ]:
# show smoothing factors are all different.
# for id in range(25):
#     print(df.iloc[499].delegators[id].smoothing_factor)